In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 369 µs (started: 2024-03-25 01:50:27 +00:00)


In [2]:
!pip install scikit-multilearn
!pip install mealpy
!pip install permetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.3 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 8.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 7.9 MB/s eta 0:00:00
time: 14.2 s (started: 2024-03-25 01:50:27 +00:00)


In [3]:
import os
import zipfile
import csv
import string
import sys
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import logging
import multiprocessing

from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2, SelectKBest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, hamming_loss
from sklearn.multiclass import OneVsRestClassifier as OVR
from sklearn.naive_bayes import MultinomialNB, ComplementNB, CategoricalNB
from sklearn.svm import LinearSVC

%matplotlib inline

time: 1.9 s (started: 2024-03-25 01:50:42 +00:00)


In [4]:
from mealpy import Optimizer, FloatVar, Multitask
from mealpy.swarm_based import GWO, ACOR

from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset

time: 56.1 ms (started: 2024-03-25 01:50:44 +00:00)


In [5]:
def get_cpu_info():
    num_cores = os.cpu_count()
    num_threads = multiprocessing.cpu_count()
    return num_cores, num_threads

cores, threads = get_cpu_info()
print(f"Number of CPU cores: {cores}, Number of CPU threads: {threads}")

Number of CPU cores: 8, Number of CPU threads: 8
time: 708 µs (started: 2024-03-25 01:50:44 +00:00)


In [6]:
# @title OriginalACOGWO Original
class OriginalACOGWO(Optimizer):
    def __init__(self, epoch: int = 10000, pop_size: int = 100, sample_count: int = 25,
                 intent_factor: float = 0.5, zeta: float = 1.0, **kwargs: object) -> None:
        """
        Args:
            epoch: maximum number of iterations, default = 10000
            pop_size: number of population size, default = 100
            sample_count: Number of Newly Generated Samples, default = 25
            intent_factor: Intensification Factor (Selection Pressure) (q in the paper), default = 0.5
            zeta: Deviation-Distance Ratio, default = 1.0
        """
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [5, 10000])
        self.sample_count = self.validator.check_int("sample_count", sample_count, [2, 10000])
        self.intent_factor = self.validator.check_float("intent_factor", intent_factor, (0, 1.0))
        self.zeta = self.validator.check_float("zeta", zeta, (0, 5))
        self.set_parameters(["epoch", "pop_size", "sample_count", "intent_factor", "zeta"])
        self.sort_flag = True

    def evolve(self, epoch):
        """
        The main operations of the ACOCGWO hybrid algorithm.
        Args:
            epoch (int): The current iteration
        """
        # Calculate ACO Selection Probabilities
        pop_rank = np.array([idx for idx in range(1, self.pop_size + 1)])
        qn = self.intent_factor * self.pop_size
        matrix_w = 1 / (np.sqrt(2 * np.pi) * qn) * np.exp(-0.5 * ((pop_rank - 1) / qn) ** 2)
        matrix_p = matrix_w / np.sum(matrix_w)  # Normalize to find the probability.

        # ACO Exploration
        matrix_pos = np.array([agent.solution for agent in self.pop])
        matrix_sigma = []
        for idx in range(0, self.pop_size):
            matrix_i = np.repeat(self.pop[idx].solution.reshape((1, -1)), self.pop_size, axis=0)
            D = np.sum(np.abs(matrix_pos - matrix_i), axis=0)
            temp = self.zeta * D / (self.pop_size - 1)
            matrix_sigma.append(temp)
        matrix_sigma = np.array(matrix_sigma)

        # Generate ACO Samples
        pop_new_aco = []
        for idx in range(0, self.sample_count):
            child = np.zeros(self.problem.n_dims)
            for jdx in range(0, self.problem.n_dims):
                rdx = self.get_index_roulette_wheel_selection(matrix_p)
                child[jdx] = self.pop[rdx].solution[jdx] + self.generator.normal() * matrix_sigma[rdx, jdx]
            pos_new_aco = self.correct_solution(child)
            agent = self.generate_empty_agent(pos_new_aco)
            pop_new_aco.append(agent)
            if self.mode not in self.AVAILABLE_MODES:
                pop_new_aco[-1].target = self.get_target(pos_new_aco)
        pop_new_aco = self.update_target_for_population(pop_new_aco)
        self.pop = self.get_sorted_and_trimmed_population(self.pop + pop_new_aco, self.pop_size, self.problem.minmax)

        # GWO Exploitation
        a = 2 - 2. * epoch / self.epoch
        _, list_best, _ = self.get_special_agents(self.pop, n_best=3, minmax=self.problem.minmax)
        pop_new_gwo = []
        for idx in range(0, self.pop_size):
            A1 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A2 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A3 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            C1 = 2 * self.generator.random(self.problem.n_dims)
            C2 = 2 * self.generator.random(self.problem.n_dims)
            C3 = 2 * self.generator.random(self.problem.n_dims)
            X1 = list_best[0].solution - A1 * np.abs(C1 * list_best[0].solution - self.pop[idx].solution)
            X2 = list_best[1].solution - A2 * np.abs(C2 * list_best[1].solution - self.pop[idx].solution)
            X3 = list_best[2].solution - A3 * np.abs(C3 * list_best[2].solution - self.pop[idx].solution)
            pos_new_gwo = (X1 + X2 + X3) / 3.0
            pos_new_gwo = self.correct_solution(pos_new_gwo)
            agent = self.generate_empty_agent(pos_new_gwo)
            pop_new_gwo.append(agent)
            if self.mode not in self.AVAILABLE_MODES:
                agent.target = self.get_target(pos_new_gwo)
                self.pop[idx] = self.get_better_agent(agent, self.pop[idx], self.problem.minmax)
        if self.mode in self.AVAILABLE_MODES:
            pop_new_gwo = self.update_target_for_population(pop_new_gwo)
            self.pop = self.greedy_selection_population(self.pop, pop_new_gwo, self.problem.minmax)

time: 3.45 ms (started: 2024-03-25 01:50:44 +00:00)


In [7]:
class ImprovedACOGWO(Optimizer):
    def __init__(self, epoch: int = 10000, pop_size: int = 100, sample_count: int = 25,
                 intent_factor: float = 0.5, zeta: float = 1.0, **kwargs: object) -> None:
        """
        Args:
            epoch: maximum number of iterations, default = 10000
            pop_size: number of population size, default = 100
            sample_count: Number of Newly Generated Samples, default = 25
            intent_factor: Intensification Factor (Selection Pressure) (q in the paper), default = 0.5
            zeta: Deviation-Distance Ratio, default = 1.0
        """
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [5, 10000])
        self.sample_count = self.validator.check_int("sample_count", sample_count, [2, 10000])
        self.intent_factor = self.validator.check_float("intent_factor", intent_factor, (0, 1.0))
        self.zeta = self.validator.check_float("zeta", zeta, (0, 5))
        self.set_parameters(["epoch", "pop_size", "sample_count", "intent_factor", "zeta"])
        self.sort_flag = True

    def evolve(self, epoch):
        """
        The main operations of the ACOCGWO hybrid algorithm.

        Args:
            epoch (int): The current iteration
        """
        # Calculate ACO Selection Probabilities
        pop_rank = np.array([idx for idx in range(1, self.pop_size + 1)])
        qn = self.intent_factor * self.pop_size
        matrix_w = 1 / (np.sqrt(2 * np.pi) * qn) * np.exp(-0.5 * ((pop_rank - 1) / qn) ** 2)
        matrix_p = matrix_w / np.sum(matrix_w)  # Normalize to find the probability.

        # ACO Exploration
        matrix_pos = np.array([agent.solution for agent in self.pop[:self.pop_size]])
        matrix_sigma = []
        for idx in range(self.pop_size):
            matrix_i = self.pop[idx].solution[np.newaxis, :] ####Numpy broadcasting
            matrix_i = np.tile(matrix_i, (self.pop_size, 1)) ######More easy understand using np.tile instead of np.repeat
            D = np.sum(np.abs(matrix_pos - matrix_i), axis=0)
            temp = self.zeta * D / (self.pop_size - 1)
            matrix_sigma.append(temp)
        matrix_sigma = np.array(matrix_sigma)

        # Generate ACO Samples
        if self.sample_count > 0:
            # random_numbers = [self.generator.normal() for _ in range(self.problem.n_dims)]
            # random_numbers = np.random.rand(self.sample_count, self.problem.n_dims)
            children = np.zeros((self.sample_count, self.problem.n_dims))
            # Generate all children at once
            for jdx in range(self.problem.n_dims):
                rdx = self.get_index_roulette_wheel_selection(matrix_p)
                children[:, jdx] = self.pop[rdx].solution[jdx] + self.generator.normal() * matrix_sigma[rdx, jdx]
            # Correct all solutions at once
            children = np.apply_along_axis(self.correct_solution, 1, children)
            # Generate all agents at once
            pop_new_aco = [self.generate_empty_agent(pos_new_aco) for pos_new_aco in children]
            if self.mode not in self.AVAILABLE_MODES:
                targets = np.apply_along_axis(self.get_target, 1, children)
                for agent, target in zip(pop_new_aco, targets):
                    agent.target = target
            pop_new_aco = self.update_target_for_population(pop_new_aco)
            self.pop[:self.pop_size] = self.get_sorted_and_trimmed_population(
                self.pop[:self.pop_size] + pop_new_aco, self.pop_size, self.problem.minmax)

        # GWO Exploitation
        a = 2 - 2. * epoch / self.epoch
        _, list_best, _ = self.get_special_agents(self.pop[:self.pop_size], n_best=3, minmax=self.problem.minmax)
        pop_new_gwo = []
        for idx in range(0, self.pop_size):
            current_solution = self.pop[idx].solution
            A1 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A2 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A3 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            C1 = 2 * self.generator.random(self.problem.n_dims)
            C2 = 2 * self.generator.random(self.problem.n_dims)
            C3 = 2 * self.generator.random(self.problem.n_dims)
            X1 = list_best[0].solution - A1 * np.abs(C1 * list_best[0].solution - current_solution)
            X2 = list_best[1].solution - A2 * np.abs(C2 * list_best[1].solution - current_solution)
            X3 = list_best[2].solution - A3 * np.abs(C3 * list_best[2].solution - current_solution)
            pos_new_gwo = (X1 + X2 + X3) / 3.0
            pos_new_gwo = self.correct_solution(pos_new_gwo)
            agent = self.generate_empty_agent(pos_new_gwo)
            pop_new_gwo.append(agent)
            if self.mode not in self.AVAILABLE_MODES:
                agent.target = self.get_target(pos_new_gwo)
                self.pop[idx] = self.get_better_agent(agent, self.pop[idx], self.problem.minmax)
        if self.mode in self.AVAILABLE_MODES:
            pop_new_gwo = self.update_target_for_population(pop_new_gwo)
            self.pop[:self.pop_size] = self.greedy_selection_population(
                self.pop[:self.pop_size], pop_new_gwo, self.problem.minmax)

time: 3.84 ms (started: 2024-03-25 01:50:44 +00:00)


In [8]:
from opfunu.cec_based.cec2014 import F12014, F12014, F22014, F32014, F42014, F52014, F62014, F72014, F82014, F92014, F102014, F112014, F122014, F132014, F142014, F152014, F162014, F172014, F182014, F192014, F202014, F212014, F222014, F232014, F242014, F252014, F262014, F272014, F282014, F292014, F302014

F12014 = F12014()
F22014 = F22014()
F32014 = F32014()
F42014 = F42014()
F52014 = F52014()
F62014 = F62014()
F72014 = F72014()
F82014 = F82014()
F92014 = F92014()
F102014 = F102014()
F112014 = F112014()
F122014 = F122014()
F132014 = F132014()
F142014 = F142014()
F152014 = F152014()
F162014 = F162014()
F172014 = F172014()
F182014 = F182014()
F192014 = F192014()
F202014 = F202014()
F212014 = F212014()
F222014 = F222014()
F232014 = F232014()
F242014 = F242014()
F252014 = F252014()
F262014 = F262014()
F272014 = F272014()
F282014 = F282014()
F292014 = F292014()
F302014 = F302014()

F12014_problem_dict = {
    "obj_func": F12014.evaluate,
    "bounds": FloatVar(lb=(-100., )*30, ub=(100., )*30),
    "minmax": "min",
    "name":"F12014",
    "obj_weights": [1],
    "save_population": True,
    "log_to": "P",
    "log_file": "F12014_history.txt"
}

def CEC2014_objective_function(x):

    return [F12014.evaluate(x), F22014.evaluate(x), F32014.evaluate(x), F42014.evaluate(x), F52014.evaluate(x), F62014.evaluate(x),
        F72014.evaluate(x), F82014.evaluate(x), F92014.evaluate(x), F102014.evaluate(x), F112014.evaluate(x), F122014.evaluate(x),
        F132014.evaluate(x), F142014.evaluate(x), F152014.evaluate(x), F162014.evaluate(x), F172014.evaluate(x), F182014.evaluate(x),
        F192014.evaluate(x), F202014.evaluate(x), F212014.evaluate(x), F222014.evaluate(x), F232014.evaluate(x), F242014.evaluate(x),
        F252014.evaluate(x), F262014.evaluate(x), F272014.evaluate(x), F282014.evaluate(x), F292014.evaluate(x), F302014.evaluate(x)]

time: 228 ms (started: 2024-03-25 01:50:44 +00:00)


In [9]:
function_count = 30
problems_list = []

for i in range(1, function_count + 1):
    function_name = f"F{i}2014"
    class_name = f"F{i}2014"

    # Assuming F12014, F22014, etc., are instances of a class
    instance = globals()[class_name]

    function_dict = {
        "obj_func": instance.evaluate,
        "bounds": FloatVar(lb=(-100., )*30, ub=(100., )*30),
        "minmax": "min",
        "name": function_name,
        "obj_weights": [1],
        "save_population": True,
        "log_to": None
    }
    problems_list.append(function_dict)

# Now function_dicts contains dictionaries for F12014 to F302014
print(problems_list[20:])

[{'obj_func': <bound method F212014.evaluate of <opfunu.cec_based.cec2014.F212014 object at 0x7f60a44bc820>>, 'bounds': <mealpy.utils.space.FloatVar object at 0x7f60a44c18b0>, 'minmax': 'min', 'name': 'F212014', 'obj_weights': [1], 'save_population': True, 'log_to': None}, {'obj_func': <bound method F222014.evaluate of <opfunu.cec_based.cec2014.F222014 object at 0x7f6070dc3100>>, 'bounds': <mealpy.utils.space.FloatVar object at 0x7f60a44c1eb0>, 'minmax': 'min', 'name': 'F222014', 'obj_weights': [1], 'save_population': True, 'log_to': None}, {'obj_func': <bound method F232014.evaluate of <opfunu.cec_based.cec2014.F232014 object at 0x7f6070df2a30>>, 'bounds': <mealpy.utils.space.FloatVar object at 0x7f60a44c1910>, 'minmax': 'min', 'name': 'F232014', 'obj_weights': [1], 'save_population': True, 'log_to': None}, {'obj_func': <bound method F242014.evaluate of <opfunu.cec_based.cec2014.F242014 object at 0x7f6070df2c40>>, 'bounds': <mealpy.utils.space.FloatVar object at 0x7f60a44c1fa0>, 'minm

In [10]:
CEC2014_problem_dict = {
    "obj_func": CEC2014_objective_function,
    "bounds": FloatVar(lb=(-100., )*30, ub=(100., )*30),
    "minmax": "min",
    "name" : "CEC2014",
    "obj_weights": [1,]*30,
}

def sphere_func(x):
    x = np.array(x).ravel()
    return np.sum(x ** 2)

problem_dict = {
    "obj_func": sphere_func,
    "bounds": FloatVar(lb=(-100., )*30, ub=(100., )*30),
    "minmax": "min",
}

time: 1.34 ms (started: 2024-03-25 01:50:44 +00:00)


In [11]:
term = {
      "max_fe": 300000,
      "epsilon": 1e-8
 }
pop_size = np.random.randint(0,30)

ImproACOGWO = ImprovedACOGWO(epoch=3000, pop_size=30, zeta=0.5, intent_factor=0.5)
ACOGWO = OriginalACOGWO(epoch=3000, pop_size=30, zeta=0.5, intent_factor=0.5)
OriGWO = GWO.OriginalGWO(epoch=3000, pop_size=30)
RWGWO = GWO.RW_GWO(epoch=3000, pop_size=30)
ACO = ACOR.OriginalACOR(epoch=3000, pop_size=30, zeta=0.5, intent_factor=0.5)

time: 1.55 ms (started: 2024-03-25 01:50:44 +00:00)


In [ ]:
multitask = Multitask(algorithms=(ACOGWO,), problems=problems_list[20:], modes=("thread", ), n_workers=8) #terminations=(term, )
multitask.execute(n_trials=5, n_jobs=None, save_path="FS_experiment/real", save_as="csv", save_convergence=True, verbose=True)

Solving problem: F212014 using algorithm: OriginalACOGWO, on the: 1 trial
Solving problem: F212014 using algorithm: OriginalACOGWO, on the: 2 trial
Solving problem: F212014 using algorithm: OriginalACOGWO, on the: 3 trial
Solving problem: F212014 using algorithm: OriginalACOGWO, on the: 4 trial
Solving problem: F212014 using algorithm: OriginalACOGWO, on the: 5 trial
Solving problem: F222014 using algorithm: OriginalACOGWO, on the: 1 trial
Solving problem: F222014 using algorithm: OriginalACOGWO, on the: 2 trial
Solving problem: F222014 using algorithm: OriginalACOGWO, on the: 3 trial
Solving problem: F222014 using algorithm: OriginalACOGWO, on the: 4 trial
Solving problem: F222014 using algorithm: OriginalACOGWO, on the: 5 trial
Solving problem: F232014 using algorithm: OriginalACOGWO, on the: 1 trial
Solving problem: F232014 using algorithm: OriginalACOGWO, on the: 2 trial
Solving problem: F232014 using algorithm: OriginalACOGWO, on the: 3 trial
Solving problem: F232014 using algorit

In [12]:
folder_to_zip = os.path.join(os.getcwd(),'FS_experiment/real')
zip_file_name = 'convergence_rate_ACOGWO.zip'
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for foldername, subfolders, filenames in os.walk(folder_to_zip):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            arcname = os.path.relpath(file_path, folder_to_zip)
            zipf.write(file_path, arcname)
zipf.close()

print(f"{zip_file_name} success zipping...")

convergence_rate_ACOGWO.zip success zipping...
time: 608 ms (started: 2024-03-25 01:51:09 +00:00)


In [ ]:
algorithm0 = Iacogwo.solve(problem=function_dicts[1], seed=42)
print(f"Best Solution: {algorithm0.solution}\n Best Fitness: {algorithm0.target.fitness}")

In [ ]:
algorithm1 = gwo.solve(problem=CEC2014_problem_dict, seed=42)
print(f"Best Solution: {algorithm1.solution}\n Best Fitness: {algorithm1.target.fitness}")

In [ ]:
algorithm2 = aco.solve(problem=F2014_problem_dict, seed=42)
print(f"Best Solution: {algorithm2.solution}\n Best Fitness: {algorithm2.target.fitness}")

In [ ]:
class ImprovedACOGWO(Optimizer):
    def __init__(self, epoch: int = 10000, pop_size: int = 100, sample_count: int = 25,
                 intent_factor: float = 0.5, zeta: float = 1.0, **kwargs: object) -> None:
        """
        Args:
            epoch: maximum number of iterations, default = 10000
            pop_size: number of population size, default = 100
            sample_count: Number of Newly Generated Samples, default = 25
            intent_factor: Intensification Factor (Selection Pressure) (q in the paper), default = 0.5
            zeta: Deviation-Distance Ratio, default = 1.0
        """
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [5, 10000])
        self.sample_count = self.validator.check_int("sample_count", sample_count, [2, 10000])
        self.intent_factor = self.validator.check_float("intent_factor", intent_factor, (0, 1.0))
        self.zeta = self.validator.check_float("zeta", zeta, (0, 5))
        self.set_parameters(["epoch", "pop_size", "sample_count", "intent_factor", "zeta"])
        self.sort_flag = True

    def evolve(self, epoch):
        """
        The main operations of the ACOCGWO hybrid algorithm.

        Args:
            epoch (int): The current iteration
        """
        # Calculate ACO Selection Probabilities
        pop_rank = np.array([idx for idx in range(1, self.pop_size + 1)])
        qn = self.intent_factor * self.pop_size
        matrix_w = 1 / (np.sqrt(2 * np.pi) * qn) * np.exp(-0.5 * ((pop_rank - 1) / qn) ** 2)
        matrix_p = matrix_w / np.sum(matrix_w)  # Normalize to find the probability.

        # ACO Exploration
        matrix_pos = np.array([agent.solution for agent in self.pop[:self.pop_size]])
        matrix_sigma = self.zeta * np.sum(np.abs(matrix_pos[:, None, :] - matrix_pos[None, :, :]), axis=0) / (self.pop_size - 1)

        # Generate ACO Samples
        if self.sample_count > 0:
          pop_new_aco = []
          random_numbers = [self.generator.normal() for _ in range(self.problem.n_dims)]
          for idx in range(0, self.sample_count):
              child = np.zeros(self.problem.n_dims)
              for jdx in range(0, self.problem.n_dims):
                  rdx = self.get_index_roulette_wheel_selection(matrix_p)
                  child[jdx] = self.pop[rdx].solution[jdx] + random_numbers[jdx] * matrix_sigma[rdx, jdx]
              pos_new_aco = self.correct_solution(child)
              agent = self.generate_empty_agent(pos_new_aco)
              pop_new_aco.append(agent)
              if self.mode not in self.AVAILABLE_MODES:
                  pop_new_aco[-1].target = self.get_target(pos_new_aco)
          pop_new_aco = self.update_target_for_population(pop_new_aco)
          self.pop[:self.pop_size] = self.get_sorted_and_trimmed_population(
              self.pop[:self.pop_size] + pop_new_aco, self.pop_size, self.problem.minmax)

        # GWO Exploitation
        a = 2 - 2. * epoch / self.epoch
        _, list_best, _ = self.get_special_agents(self.pop[:self.pop_size], n_best=3, minmax=self.problem.minmax)
        pop_new_gwo = []
        for idx in range(0, self.pop_size):
            current_solution = self.pop[idx].solution
            A1 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A2 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            A3 = a * (2 * self.generator.random(self.problem.n_dims) - 1)
            C1 = 2 * self.generator.random(self.problem.n_dims)
            C2 = 2 * self.generator.random(self.problem.n_dims)
            C3 = 2 * self.generator.random(self.problem.n_dims)
            X1 = list_best[0].solution - A1 * np.abs(C1 * list_best[0].solution - current_solution)
            X2 = list_best[1].solution - A2 * np.abs(C2 * list_best[1].solution - current_solution)
            X3 = list_best[2].solution - A3 * np.abs(C3 * list_best[2].solution - current_solution)
            pos_new_gwo = (X1 + X2 + X3) / 3.0
            pos_new_gwo = self.correct_solution(pos_new_gwo)
            agent = self.generate_empty_agent(pos_new_gwo)
            pop_new_gwo.append(agent)
            if self.mode not in self.AVAILABLE_MODES:
                agent.target = self.get_target(pos_new_gwo)
                self.pop[idx] = self.get_better_agent(agent, self.pop[idx], self.problem.minmax)
        if self.mode in self.AVAILABLE_MODES:
            pop_new_gwo = self.update_target_for_population(pop_new_gwo)
            self.pop[:self.pop_size] = self.greedy_selection_population(
                self.pop[:self.pop_size], pop_new_gwo, self.problem.minmax)